# Deep Learning

In [1]:
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import regularizers
import keras
import pandas as pd
import numpy as np
from keras import backend as K
from keras import metrics
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

Using TensorFlow backend.


In [2]:
kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")

#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
#y_test_labels = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")

output_columns_2labels = ['is_Attack','is_Normal']

from sklearn import model_selection as ms
from sklearn import preprocessing as pp

x_input = kdd_train_2labels.drop(output_columns_2labels, axis = 1)
y_output = kdd_train_2labels.loc[:,output_columns_2labels]

ss = pp.StandardScaler()
x_input = ss.fit_transform(x_input)

#le = pp.LabelEncoder()
#y_train = le.fit_transform(y_train_labels).reshape(-1, 1)
#y_test = le.transform(y_test_labels).reshape(-1, 1)

y_train = kdd_train_2labels.loc[:,output_columns_2labels]

x_train, x_valid, y_train, y_valid = ms.train_test_split(x_input, 
                              y_train, 
                              test_size=0.1)
#x_valid, x_test, y_valid, y_test = ms.train_test_split(x_valid, y_valid, test_size = 0.4)

x_test = kdd_test_2labels.drop(output_columns_2labels, axis = 1)
y_test = kdd_test_2labels.loc[:,output_columns_2labels]

x_test = ss.transform(x_test)

#x_train = np.hstack((x_train, y_train))
#x_valid = np.hstack((x_valid, y_valid))

#x_test = np.hstack((x_test, np.random.normal(loc = 0, scale = 0.01, size = y_test.shape)))

In [3]:
input_dim = 122
intermediate_dim = 122
latent_dim = 32
batch_size = 1409
epochs = 5
hidden_layers = 8

class Train:
    def train():
        Train.x = Input(shape=(input_dim,))
        
        hidden_encoder = Train.x
        for i in range(hidden_layers):
            hidden_encoder = Dense(intermediate_dim, activation='relu')(hidden_encoder)

        Train.mean_encoder = Dense(latent_dim, activation=None)(hidden_encoder)
        Train.logvar_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        def get_distrib(args):

            m_e, l_e = args

            # Sample epsilon
            epsilon = np.random.normal(loc=0.0, scale=0.05, size = (batch_size, latent_dim))

            # Sample latent variable
            z = m_e + K.exp(l_e / 2) * epsilon
            return z

        z = Lambda(get_distrib)([Train.mean_encoder, Train.logvar_encoder])

        hidden_decoder = z
        for i in range(hidden_layers):
            hidden_decoder = Dense(intermediate_dim, activation="relu")(hidden_decoder)

        Train.x_ = Dense(input_dim, activation=None)(hidden_decoder)


In [4]:
import itertools
#features_arr = [4, 16, 32, 256, 1024]
#hidden_layers_arr = [2, 6, 10, 100]

features_arr = [2, 4, 8, 16, 32]
hidden_layers_arr = [2, 6, 10]

epoch_arr = [50]

score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
scores = []
predictions = pd.DataFrame()

for e, h, f in itertools.product(epoch_arr, hidden_layers_arr, features_arr):
    
    print(" \n Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
    latent_dim = f
    epochs = e
    hidden_layers = h

    Train.train()

    optimizer = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-04, decay=0.1)
    vae_model = Model(inputs = Train.x, outputs = Train.x_ )
    vae_model.compile(optimizer = optimizer, loss = 'mse', metrics = ['accuracy'] )

    train_size = x_train.shape[0] - x_train.shape[0]%batch_size
    valid_size = x_valid.shape[0] - x_valid.shape[0]%batch_size

    vae_model.fit(x = x_train[:train_size,:], y = x_train[:train_size,:], 
                  shuffle=True, epochs=epochs, 
                  batch_size = batch_size, 
                  validation_data = (x_test, x_test),
                  verbose = 1)
    
    score_train = vae_model.evaluate(x_valid[:valid_size,:], y = x_valid[:valid_size,:],
                               batch_size = batch_size,
                               verbose = 1)
    
    score_test = vae_model.evaluate(x_test, y = x_test,
                           batch_size = batch_size,
                           verbose = 1)
    
    scores.append(score(e,f,h,score_train[-1], score_test[-1])) #score_test[-1]))
    
    print("\n Train Acc: {}, Test Acc: {}".format(score_train[-1], 
                                                  score_test[-1])  )
    
scores = pd.DataFrame(scores)


 
 Current Layer Attributes - epochs:50 hidden layers:2 features count:2
Train on 112720 samples, validate on 22544 samples
Epoch 1/50
112720/112720 [==============================] - 1s - loss: 0.9199 - acc: 0.0269 - val_loss: 1.8596 - val_acc: 0.0625
Epoch 2/50
112720/112720 [==============================] - 0s - loss: 582141.9507 - acc: 0.0783 - val_loss: 1.8115 - val_acc: 0.0812
Epoch 3/50
112720/112720 [==============================] - 0s - loss: 0.8251 - acc: 0.0915 - val_loss: 1.8051 - val_acc: 0.0964
Epoch 4/50
112720/112720 [==============================] - 0s - loss: 0.8081 - acc: 0.0959 - val_loss: 1.7915 - val_acc: 0.0837
Epoch 5/50
112720/112720 [==============================] - 0s - loss: 0.7937 - acc: 0.0984 - val_loss: 1.7811 - val_acc: 0.0872
Epoch 6/50
112720/112720 [==============================] - 0s - loss: 0.7851 - acc: 0.0991 - val_loss: 1.7735 - val_acc: 0.0931
Epoch 7/50
112720/112720 [==============================] - 0s - loss: 0.7788 - acc: 0.1232 - val

112720/112720 [==============================] - 0s - loss: 0.7234 - acc: 0.4397 - val_loss: 1.7343 - val_acc: 0.4465
Epoch 12/50
112720/112720 [==============================] - 0s - loss: 0.7184 - acc: 0.4461 - val_loss: 1.7310 - val_acc: 0.4559
Epoch 13/50
112720/112720 [==============================] - 0s - loss: 0.7138 - acc: 0.4608 - val_loss: 1.7283 - val_acc: 0.4692
Epoch 14/50
112720/112720 [==============================] - 1s - loss: 0.7096 - acc: 0.4798 - val_loss: 1.7253 - val_acc: 0.4835
Epoch 15/50
112720/112720 [==============================] - 1s - loss: 0.7057 - acc: 0.4951 - val_loss: 1.7221 - val_acc: 0.4816
Epoch 16/50
112720/112720 [==============================] - 1s - loss: 0.7020 - acc: 0.5081 - val_loss: 1.7197 - val_acc: 0.4924
Epoch 17/50
112720/112720 [==============================] - 1s - loss: 0.6985 - acc: 0.5209 - val_loss: 1.7172 - val_acc: 0.4914
Epoch 18/50
112720/112720 [==============================] - 1s - loss: 0.6953 - acc: 0.5296 - val_los

112720/112720 [==============================] - 0s - loss: 0.4805 - acc: 0.7400 - val_loss: 1.2844 - val_acc: 0.7693
Epoch 23/50
112720/112720 [==============================] - 0s - loss: 0.4762 - acc: 0.7416 - val_loss: 1.2753 - val_acc: 0.7713
Epoch 24/50
112720/112720 [==============================] - 0s - loss: 0.4722 - acc: 0.7451 - val_loss: 1.2650 - val_acc: 0.7716
Epoch 25/50
112720/112720 [==============================] - 0s - loss: 0.4684 - acc: 0.7461 - val_loss: 1.2566 - val_acc: 0.7733
Epoch 26/50
112720/112720 [==============================] - 0s - loss: 0.4647 - acc: 0.7477 - val_loss: 1.2476 - val_acc: 0.7740
Epoch 27/50
112720/112720 [==============================] - 0s - loss: 0.4612 - acc: 0.7501 - val_loss: 1.2390 - val_acc: 0.7755
Epoch 28/50
112720/112720 [==============================] - 0s - loss: 0.4580 - acc: 0.7509 - val_loss: 1.2305 - val_acc: 0.7757
Epoch 29/50
112720/112720 [==============================] - 0s - loss: 0.4548 - acc: 0.7513 - val_los

112720/112720 [==============================] - 0s - loss: 0.6164 - acc: 0.6612 - val_loss: 1.5615 - val_acc: 0.6464
Epoch 34/50
112720/112720 [==============================] - 0s - loss: 0.6149 - acc: 0.6637 - val_loss: 1.5600 - val_acc: 0.6467
Epoch 35/50
112720/112720 [==============================] - 0s - loss: 0.6134 - acc: 0.6641 - val_loss: 1.5581 - val_acc: 0.6470
Epoch 36/50
112720/112720 [==============================] - 0s - loss: 0.6120 - acc: 0.6648 - val_loss: 1.5559 - val_acc: 0.6475
Epoch 37/50
112720/112720 [==============================] - 0s - loss: 0.6106 - acc: 0.6652 - val_loss: 1.5539 - val_acc: 0.6482
Epoch 38/50
112720/112720 [==============================] - 0s - loss: 0.6093 - acc: 0.6665 - val_loss: 1.5521 - val_acc: 0.6484
Epoch 39/50
112720/112720 [==============================] - 0s - loss: 0.6079 - acc: 0.6671 - val_loss: 1.5504 - val_acc: 0.6489
Epoch 40/50
112720/112720 [==============================] - 0s - loss: 0.6066 - acc: 0.6693 - val_los

112720/112720 [==============================] - 1s - loss: nan - acc: 0.0050 - val_loss: nan - val_acc: 1.7743e-04
Epoch 45/50
112720/112720 [==============================] - 1s - loss: nan - acc: 0.0050 - val_loss: nan - val_acc: 1.7743e-04
Epoch 46/50
112720/112720 [==============================] - 1s - loss: nan - acc: 0.0050 - val_loss: nan - val_acc: 1.7743e-04
Epoch 47/50
112720/112720 [==============================] - 1s - loss: nan - acc: 0.0050 - val_loss: nan - val_acc: 1.7743e-04
Epoch 48/50
112720/112720 [==============================] - 1s - loss: nan - acc: 0.0050 - val_loss: nan - val_acc: 1.7743e-04
Epoch 49/50
112720/112720 [==============================] - 1s - loss: nan - acc: 0.0050 - val_loss: nan - val_acc: 1.7743e-04
Epoch 50/50
14090/22544 [=================>............] - ETA: 0s
 Train Acc: 0.005145493312738836, Test Acc: 0.00017743080388754606
 
 Current Layer Attributes - epochs:50 hidden layers:6 features count:2
Train on 112720 samples, validate on 

...

112720/112720 [==============================] - 2s - loss: 0.7056 - acc: 0.1775 - val_loss: 1.7222 - val_acc: 0.1092
Epoch 37/50
112720/112720 [==============================] - 2s - loss: 0.7038 - acc: 0.1822 - val_loss: 1.7213 - val_acc: 0.1132
Epoch 38/50
112720/112720 [==============================] - 2s - loss: 0.7045 - acc: 0.1867 - val_loss: 1.7190 - val_acc: 0.1164
Epoch 39/50
112720/112720 [==============================] - 2s - loss: 0.7031 - acc: 0.1916 - val_loss: 1.7164 - val_acc: 0.1263
Epoch 40/50
112720/112720 [==============================] - 2s - loss: 0.7011 - acc: 0.1963 - val_loss: 1.7132 - val_acc: 0.1337
Epoch 41/50
112720/112720 [==============================] - 2s - loss: 0.6995 - acc: 0.2008 - val_loss: 1.7134 - val_acc: 0.1467
Epoch 42/50
112720/112720 [==============================] - 2s - loss: 0.6973 - acc: 0.2060 - val_loss: 1.7102 - val_acc: 0.1566
Epoch 43/50
112720/112720 [==============================] - 2s - loss: 0.6975 - acc: 0.2127 - val_los

112720/112720 [==============================] - 3s - loss: 0.6402 - acc: 0.7068 - val_loss: 1.6883 - val_acc: 0.6739
Epoch 48/50
112720/112720 [==============================] - 3s - loss: 0.6389 - acc: 0.7087 - val_loss: 1.6859 - val_acc: 0.6748
Epoch 49/50
112720/112720 [==============================] - 3s - loss: 0.6380 - acc: 0.7101 - val_loss: 1.6846 - val_acc: 0.6767
Epoch 50/50
22544/22544 [==============================] - 0s     

 Train Acc: 0.715312272310257, Test Acc: 0.6772090159356594
 
 Current Layer Attributes - epochs:50 hidden layers:10 features count:4
Train on 112720 samples, validate on 22544 samples
Epoch 1/50
112720/112720 [==============================] - 5s - loss: 0.9806 - acc: 0.0056 - val_loss: 1.9554 - val_acc: 0.0014
Epoch 2/50
112720/112720 [==============================] - 3s - loss: 0.9076 - acc: 0.0011 - val_loss: 1.9080 - val_acc: 0.0019
Epoch 3/50
112720/112720 [==============================] - 3s - loss: 0.8646 - acc: 0.0031 - val_loss: 1.8561 

112720/112720 [==============================] - 3s - loss: 0.8308 - acc: 0.0111 - val_loss: 1.8451 - val_acc: 0.0205
Epoch 7/50
112720/112720 [==============================] - 3s - loss: 0.8229 - acc: 0.0119 - val_loss: 1.8376 - val_acc: 0.0089
Epoch 8/50
112720/112720 [==============================] - 3s - loss: 0.8131 - acc: 0.0449 - val_loss: 1.8291 - val_acc: 0.2878
Epoch 9/50
112720/112720 [==============================] - 3s - loss: 0.8038 - acc: 0.3386 - val_loss: 1.8158 - val_acc: 0.3585
Epoch 10/50
112720/112720 [==============================] - 3s - loss: 0.7949 - acc: 0.4003 - val_loss: 1.8148 - val_acc: 0.4237
Epoch 11/50
112720/112720 [==============================] - 3s - loss: 0.7850 - acc: 0.4214 - val_loss: 1.8065 - val_acc: 0.4561
Epoch 12/50
112720/112720 [==============================] - 3s - loss: 0.7745 - acc: 0.4293 - val_loss: 1.7903 - val_acc: 0.4608
Epoch 13/50
112720/112720 [==============================] - 3s - loss: 0.7649 - acc: 0.4382 - val_loss: 

112720/112720 [==============================] - 3s - loss: 0.7608 - acc: 0.5161 - val_loss: 1.7845 - val_acc: 0.5749
Epoch 17/50
112720/112720 [==============================] - 3s - loss: 0.7534 - acc: 0.5346 - val_loss: 1.7714 - val_acc: 0.5808
Epoch 18/50
112720/112720 [==============================] - 3s - loss: 0.7473 - acc: 0.5405 - val_loss: 1.7587 - val_acc: 0.5803
Epoch 19/50
112720/112720 [==============================] - 3s - loss: 0.7390 - acc: 0.5462 - val_loss: 1.7522 - val_acc: 0.5821
Epoch 20/50
112720/112720 [==============================] - 3s - loss: 0.7325 - acc: 0.5483 - val_loss: 1.7359 - val_acc: 0.5836
Epoch 21/50
112720/112720 [==============================] - 3s - loss: 0.7261 - acc: 0.5511 - val_loss: 1.7286 - val_acc: 0.5861
Epoch 22/50
112720/112720 [==============================] - 3s - loss: 0.7205 - acc: 0.5526 - val_loss: 1.7238 - val_acc: 0.5915
Epoch 23/50
112720/112720 [==============================] - 3s - loss: 0.7153 - acc: 0.5562 - val_los

112720/112720 [==============================] - 3s - loss: 0.7373 - acc: 0.3579 - val_loss: 1.7667 - val_acc: 0.4181
Epoch 27/50
112720/112720 [==============================] - 3s - loss: 0.7355 - acc: 0.3592 - val_loss: 1.7654 - val_acc: 0.4237
Epoch 28/50
112720/112720 [==============================] - 2s - loss: 0.7338 - acc: 0.3629 - val_loss: 1.7643 - val_acc: 0.4233
Epoch 29/50
112720/112720 [==============================] - 2s - loss: 0.7323 - acc: 0.3700 - val_loss: 1.7638 - val_acc: 0.4295
Epoch 30/50
112720/112720 [==============================] - 2s - loss: 0.7310 - acc: 0.3800 - val_loss: 1.7628 - val_acc: 0.4330
Epoch 31/50
112720/112720 [==============================] - 3s - loss: 0.7297 - acc: 0.3913 - val_loss: 1.7623 - val_acc: 0.4380
Epoch 32/50
112720/112720 [==============================] - 3s - loss: 0.7285 - acc: 0.4048 - val_loss: 1.7618 - val_acc: 0.4530
Epoch 33/50
112720/112720 [==============================] - 3s - loss: 0.7274 - acc: 0.4156 - val_los

In [5]:
scores.sort_values("test_score", ascending=False)

,epoch,no_of_features,hidden_layers,train_score,test_score
6,50,4,6,0.853620,0.795156
2,50,8,2,0.755500,0.786329
8,50,16,6,0.727289,0.744056
10,50,2,10,0.715312,0.677209
3,50,16,2,0.666519,0.653478
11,50,4,10,0.581796,0.642566
5,50,2,6,0.600869,0.636267
1,50,4,2,0.595635,0.631609
13,50,16,10,0.581707,0.630101
0,50,2,2,0.558286,0.622826


In [6]:
scores.to_pickle("dataset/vae_only_feature_extraction_scores.pkl")